# Numbers from the catalogs and photometry
Here we compute the numbers that we obtain from the photometry and the selected samples from VCV and SMB.

First we import all necessary packages

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table, vstack, join

In [3]:
def SelT1orT2(TableP):
    """Select Seyferts 1 and 2 in both VCV and CDS"""
    Selc1 = np.logical_and(TableP['otype_txt'] == 'Sy1', TableP['Sp'] == 'S1')
    Selc2 = np.logical_and(TableP['otype_txt'] == 'Sy2', TableP['Sp'] == 'S2')
    SelcSey = np.logical_or(Selc1, Selc2)
    return(TableP[SelcSey])


def SelT1xorT2(TableP):
    """Select mixed classifications Seyferts 1 and 2 between VCV and CDS"""
    Selc1 = np.logical_and(TableP['otype_txt'] == 'Sy1', TableP['Sp'] == 'S2')
    Selc2 = np.logical_and(TableP['otype_txt'] == 'Sy2', TableP['Sp'] == 'S1')
    SelcSey = np.logical_or(Selc1, Selc2)
    return(TableP[SelcSey])

## Numbers in photometry

Here we examine the photometry of the all galaxies we decided to use for CIGALE runs.

In [4]:
Phot = Table.read('CIGPhot_EnergyBal_All.tbl', format='ascii').to_pandas()

The counts of the total bands in the photometry show that most information comes from SDSS and WISE, and we have a low coverage in the longer wavelengths. This will define the TABLE 1 in our work

In [5]:
Phot.describe().T['count'][1::2] 

FUV            6476.0
NUV            9258.0
u_prime       11943.0
g_prime       12459.0
r_prime       12239.0
i_prime       12190.0
z_prime       11520.0
J_2mass        7259.0
H_2mass        6892.0
Ks_2mass       8339.0
WISE1         13126.0
IRAC1          4022.0
IRAC2          4009.0
WISE2         13118.0
IRAC3           398.0
IRAC4           390.0
IRAS1           446.0
WISE3         12188.0
WISE4          8076.0
MIPS1           714.0
IRAS2           608.0
IRAS3           914.0
PACS_blue       244.0
MIPS2           213.0
IRAS4           694.0
PACS_green      159.0
MIPS3           103.0
PACS_red        285.0
PSW_ext         721.0
PMW_ext         458.0
PLW_ext         214.0
Name: count, dtype: float64

## Numbers in catalogs (VCV)

We use the file where galaxies has been classified as Seyfert by VCV and also show a Seyfert type in SIMBAD.

In [6]:
TV = Table.read('VCV_TAP_otype.txt', format='ascii').to_pandas()

First we cound how many galaxies are for each type. We assumed that S1n are also S1 galaxies. Most of the galaxies are Seyfert 1 and Seyfert 2 ({{sum(TV.groupby('Sp').size()[['S1', 'S1n']])}}, {{sum(TV.groupby('Sp').size()[['S2']])}}), while the unclasified Seyfert are {{sum(TV.groupby('Sp').size()[['S']])}}. The total number of galaxies outside the normal classification are {{sum(TV.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9','S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']])}}, where we do not use {{sum(TV.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']])}} of those galaxies when VCV is compared. 

In [7]:
print('Classified Seyfer galaxies:',
      sum(TV.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications',
      sum(TV.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications',
      sum(TV.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']]))
TV.groupby('Sp').size()

Classified Seyfer galaxies: 17831
Intermediate classifications 920
Other classifications 170


Sp
S          84
S1      10936
S1.0      162
S1.2      161
S1.5      356
S1.8      114
S1.9      127
S1h        40
S1i         5
S1n      2244
S1n0        7
S1n2        5
S1n5        5
S2       4567
S3         58
S3b         6
S?         44
dtype: int64

## Numbers in catalogs (SMB)

In the SIMBAD type we have {{sum(TV.groupby('otype_txt').size()[['Sy1']])}} Seyfert 1 and {{sum(TV.groupby('otype_txt').size()[['Sy2']])}} Seyfert 2, with {{sum(TV.groupby('otype_txt').size()[['SyG']])}} unclassified Seyfert. More information can be found in Clean_Sample notebook.

In [8]:
TV.groupby('otype_txt').size()

otype_txt
Sy1    13760
Sy2     5040
SyG      121
dtype: int64

## Numbers on Classifications with SMB and VCV

From the {{len(TV)}} galaxies, a total of {{len(SelT1orT2(TV))}} are classified in S1 or S2 in both catalogs, and {{len(SelT1xorT2(TV))}} galaxies change of type (S1 -> S2 or S2 -> S1). 

In [9]:
len(SelT1orT2(TV)), len(SelT1xorT2(TV))

(15036, 454)

Now we can count the number of number of galaxies for each classification type as before for the VCV and SMB.

In [10]:
PhotTV = Phot.join(TV.set_index('main_id'), on='id')

In [11]:
print('Classified Seyfer galaxies:',
      sum(PhotTV.groupby('Sp').size()[['S1', 'S1n', 'S2', 'S']]))
print('Intermediate classifications',
      sum(PhotTV.groupby('Sp').size()[['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9']]))
print('Other classifications',
      sum(PhotTV.groupby('Sp').size()[['S1h', 'S1i', 'S1n0', 'S1n2', 'S1n5', 'S3', 'S3b', 'S?']]))
PhotTV.groupby('Sp').size()

Classified Seyfer galaxies: 12232
Intermediate classifications 760
Other classifications 137


Sp
S         55
S1      7013
S1.0     151
S1.2     133
S1.5     294
S1.8      90
S1.9      92
S1h       36
S1i        4
S1n     1905
S1n0       7
S1n2       5
S1n5       5
S2      3259
S3        49
S3b        5
S?        26
dtype: int64

In [12]:
PhotTV.groupby('otype_txt').size()

otype_txt
Sy1    9400
Sy2    3655
SyG      74
dtype: int64

In [13]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy,pandas

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.2
pandas : 1.2.0

re        : 2.2.1
numpy     : 1.19.5
matplotlib: 3.2.2
json      : 2.0.9
logging   : 0.5.1.2
autopep8  : 1.5.4
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]

Watermark: 2.1.0

